# **Tiền xử lý dữ liệu**
Trong bước này, chúng ta sẽ làm sạch và chuẩn hóa dữ liệu thô để sẵn sàng cho việc phân tích và mô hình hóa. Các nhiệm vụ chính bao gồm:
- Loại bỏ các cột như đã phân tích ở file `02_exploration.ipynb`
- Chuẩn hóa định dạng thời gian
- Xử lý dữ liệu bị thiếu (`fatality_count`, `injury_count`, `admin_division_population`)
- Chuẩn hoá cơ bản các cột text
- Làm sạch dữ liệu văn bản và phân loại.
- Kiểm tra tính hợp lệ của dữ liệu địa lý.

**Import các thư viện cần thiết**

In [7]:
import pandas as pd
import numpy as np
import re # Thư viện xử lý biểu thức chính quy (Regex) cho văn bản
import os


import sys
sys.path.append(os.path.abspath(r"..\src"))

import config as cf
import data_processing as dp 
import utils as ut 

**Load dữ liệu**

In [8]:
df = pd.read_csv(cf.RAW_DATA)
print(f"Dữ liệu gốc có kích thước: {df.shape}")
df.head(3)

Dữ liệu gốc có kích thước: (11033, 31)


,source_name,source_link,event_id,event_date,event_time,event_title,event_description,location_description,location_accuracy,landslide_category,...,country_code,admin_division_name,admin_division_population,gazeteer_closest_point,gazeteer_distance,submitted_date,created_date,last_edited_date,longitude,latitude
0,AGU,https://blogs.agu.org/landslideblog/2008/10/14...,684,08/01/2008 12:00:00 AM,NaN,"Sigou Village, Loufan County, Shanxi Province","occurred early in morning, 11 villagers buried...","Sigou Village, Loufan County, Shanxi Province",unknown,landslide,...,CN,Shaanxi,0.0,Jingyang,41.02145,04/01/2014 12:00:00 AM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,107.4500,32.5625
1,Oregonian,http://www.oregonlive.com/news/index.ssf/2009/...,956,01/02/2009 02:00:00 AM,NaN,"Lake Oswego, Oregon",Hours of heavy rain are to blame for an overni...,"Lake Oswego, Oregon",5km,mudslide,...,US,Oregon,36619.0,Lake Oswego,0.60342,04/01/2014 12:00:00 AM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,-122.6630,45.4200
2,CBS News,https://www.cbsnews.com/news/dozens-missing-af...,973,01/19/2007 12:00:00 AM,NaN,"San Ramon district, 195 miles northeast of the...",(CBS/AP) At least 10 people died and as many a...,"San Ramon district, 195 miles northeast of the...",10km,landslide,...,PE,Junín,14708.0,San Ramón,0.85548,04/01/2014 12:00:00 AM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,-75.3587,-11.1295


## **1. Loại bỏ các cột không cần thiết**
Dựa trên phân tích từ bước `02_data_exploration`, chúng ta sẽ loại bỏ các cột chứa quá nhiều giá trị null hoặc không mang lại giá trị cho việc phân tích.

In [9]:
cols_to_drop = [
    'event_id',
    'event_import_id', 
    'event_import_source', 
    'created_date', 
    'submitted_date', 
    'last_edited_date',
    'source_link', 
    'photo_link', 
    'storm_name',
    'country_code', 
    'event_time', 
    'notes',
    'gazeteer_closest_point', 
    'gazeteer_distance'
]

# Xóa cột 
df.drop(columns=[c for c in cols_to_drop if c in df.columns], inplace=True)

print(f"Kích thước sau khi xóa cột thừa: {df.shape}")


Kích thước sau khi xóa cột thừa: (11033, 17)


## **2. Sửa lỗi hiển thị văn bản**
Trong bộ dữ liệu, có một số cột văn bản bị lỗi mã hóa ký tự, dẫn đến việc hiển thị các ký tự đặc biệt không đúng. Để khắc phục điều này, ta sẽ áp dụng phương pháp giải mã và mã hóa lại các chuỗi văn bản trong các cột bị ảnh hưởng. 

In [10]:
cols_to_fix = ['event_description', 'location_description', 
               'admin_division_name', 'event_title','source_name', 'admin_division_name', 'country_name']
for col in cols_to_fix:
    df[col] = df[col].apply(ut.fix_encoding)

print("Đã sửa xong lỗi font cho các cột văn bản.")

Đã sửa xong lỗi font cho các cột văn bản.


##  **3. Xử lý Dữ liệu Thời gian**
Cột `event_date` rất quan trọng để phân tích xu hướng. Chúng ta cần chuyển nó sang định dạng datetime. 

In [11]:
# Chuyển đổi sang datetime, các giá trị lỗi sẽ biến thành NaT
df['event_date'] = pd.to_datetime(df['event_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

print("Đã hoàn thành xử lý thời gian.")

Đã hoàn thành xử lý thời gian.


## **4. xử lý Missing**
### **4.1 `event_description`**

**Phương pháp:** Ta sẽ lấy nội dung từ `event_title` nếu `event_description` bị thiếu, sau đó nếu vẫn thiếu thì gán là `unknown` .

In [12]:
# Xử lý 'event_description'
df['event_description'] = df['event_description'].fillna(df['event_title']).fillna('unknown')

df['event_description'] = df['event_description'].astype(str)

print(f"Đã xử lý xong dữ liệu văn bản thiếu.")

Đã xử lý xong dữ liệu văn bản thiếu.


###  **4.2 `fatality_count` và `injury_count`**

**Phương pháp:** Chúng ta sẽ cố gắng trích xuất thông tin từ cột `event_description` để điền vào các giá trị này. Nếu như không tìm thấy dữ liệu ở cột `event_description` thì sẽ  gán bằng giá trị 0 

**Lưu ý:** Việc điền giá trị bằng `0` như vậy sẽ khiến giá trị bị lệch về phía 0 (đồng thời ở trường `injury_count` cũng bị thiếu rất nhiều, $\approx{51\%}$, việc điền như vậy sẽ làm mất đi tính thực tế của dữ liệu (missing != `0`)), tuy nhiên trong trường hợp này ta chấp nhận điều đó vì không có cách nào khác để xác định được giá trị thực tế.



In [13]:
# Tạo danh sách descriptions
descriptions = df['event_description'].astype(str).tolist()

# Trích xuất thông tin từ event_description bằng hàm extract_casualties
extraction_result = [dp.extract_casualties(desc) for desc in descriptions]
extraction_result = pd.Series(extraction_result, index=df.index)

extracted_fatalities = pd.to_numeric(extraction_result.apply(lambda x: x[0] if isinstance(x, tuple) and len(x) > 0 else None), errors='coerce')
extracted_injuries = pd.to_numeric(extraction_result.apply(lambda x: x[1] if isinstance(x, tuple) and len(x) > 1 else None), errors='coerce')


# Đảm bảo cột gốc cũng là số để so sánh
df['fatality_count'] = pd.to_numeric(df['fatality_count'], errors='coerce')
df['injury_count'] = pd.to_numeric(df['injury_count'], errors='coerce')

# Xử lý fatality_count - chỉ điền nếu như cột gốc là NaN
update_mask_fat = extracted_fatalities.notna() & (
    df['fatality_count'].isna())

df.loc[update_mask_fat, 'fatality_count'] = extracted_fatalities
df.loc[update_mask_fat, 'fatality_imputed'] = True


# Xử lý injury_count - chỉ điền nếu như cột gốc là NaN
update_mask_inj = extracted_injuries.notna() & (
    df['injury_count'].isna() )

df.loc[update_mask_inj, 'injury_count'] = extracted_injuries
df.loc[update_mask_inj, 'injury_imputed'] = True


# Hoàn tất việc tạo cờ - những dòng còn lại sẽ là NaN, ta fill False
df['fatality_imputed'] = df['fatality_imputed'].fillna(False).astype(bool)
df['injury_imputed'] = df['injury_imputed'].fillna(False).astype(bool)


# Điền giá trị 0 cho các dòng vẫn còn NaN ở cột fatality_count và injury_count
df['fatality_count'] = df['fatality_count'].fillna(0).astype(int)
df['injury_count'] = df['injury_count'].fillna(0).astype(int)


num_updated_fatality = df['fatality_imputed'].sum()
num_updated_injury = df['injury_imputed'].sum()

print(f"\n[Kết quả Smart Imputation & Correction]")
print(f"Đã tự động điền {num_updated_fatality} dòng số người chết.")
print(f"Đã tự động điền {num_updated_injury} dòng số người bị thương.")


[Kết quả Smart Imputation & Correction]
Đã tự động điền 62 dòng số người chết.
Đã tự động điền 335 dòng số người bị thương.


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8976\585211124.py:33: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['fatality_imputed'] = df['fatality_imputed'].fillna(False).astype(bool)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8976\585211124.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['injury_imputed'] = df['injury_imputed'].fillna(False).astype(bool)


###  **4.2 `admin_division_population`**

**Phương pháp:** Chúng ta sẽ điền giá trị bị thiếu trong cột `admin_division_population` bằng cách sử dụng giá trị trung vị (không dùng trung bình) của các vùng hành chính tương tự (cùng quốc gia), ta cũng sẽ thay thế các giá trị 0 bằng NaN để dễ xử lý, và nếu một quốc gia có toàn bộ giá trị bị thiếu, ta sẽ sử dụng giá trị trung vị toàn cầu để điền vào.

In [14]:

# Thay số 0 bằng NaN để dễ xử lý chung với các ô trống
df['admin_division_population'] = df['admin_division_population'].replace(0, np.nan)

# Điền NaN bằng giá trị trung vị (median) của TỪNG QUỐC GIA
# Nếu quốc gia đó cũng toàn NaN, thì mới dùng trung vị toàn cầu
global_median = df['admin_division_population'].median()

df['admin_division_population'] = df.groupby('country_name')['admin_division_population'].transform(
    lambda x: x.fillna(x.median())
)

# Điền nốt những ô vẫn còn NaN bằng global median
df['admin_division_population'] = df['admin_division_population'].fillna(global_median)

# Đảm bảo kiểu dữ liệu là float hoặc int
df['admin_division_population'] = df['admin_division_population'].astype(float)

print(f"Đã xử lý dân số = 0 và NaN bằng Median theo quốc gia.")

Đã xử lý dân số = 0 và NaN bằng Median theo quốc gia.


d:\Study\HCMUS\2025-2026_HK1\Lập trình cho KHDL\Final Project\Global-Landslide-Catalog-NASA-Analysis\vevn\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
d:\Study\HCMUS\2025-2026_HK1\Lập trình cho KHDL\Final Project\Global-Landslide-Catalog-NASA-Analysis\vevn\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


### **4.3 `admin_division_name` và `country_name`**

**Phương pháp:** Sử dụng kỹ thuật Reverse Geocoding để lấy thông tin địa lý từ tọa độ (latitude, longitude). Nếu không thể lấy được thông tin, ta sẽ giữ nguyên giá trị hiện tại.

In [15]:
df = df.apply(dp.fill_missing_locations, axis=1) # hàm điền thiếu admin_division_name, country_name từ tọa độ
print("Hoàn tất.")

Hoàn tất.


### **4.4 Điền `unknown` nếu còn lại các ô trống**

In [16]:
# Danh sách các cột phân loại cần đảm bảo không có giá trị thiếu
cols_to_fill_unknown = [
    'country_name', 
    'admin_division_name',
    'location_description',
    'landslide_trigger',
    'landslide_setting',
    'landslide_category',
    'event_description',
    'event_title'
]

for col in cols_to_fill_unknown:
    # Bước 1: Đảm bảo cột là kiểu string/object để chứa 'unknown'
    df[col] = df[col].astype(str)
    
    # Bước 2: Điền NaN thực sự (nếu còn) bằng chuỗi rỗng
    df[col] = df[col].fillna('')
    
    # Bước 3: Thay thế chuỗi rỗng "" (từ các bước làm sạch) bằng 'unknown'
    df.loc[df[col] == '', col] = 'unknown'
    
    # Bước 4 (Kiểm tra): Đảm bảo các giá trị 'nan' (chuỗi) cũng được chuẩn hóa
    df.loc[df[col].str.lower().str.strip() == 'nan', col] = 'unknown'

    df[col] = df[col].apply(dp.clean_text) # clean text cơ bản

print("Đã hoàn thành điền thiếu 'unknown' cho tất cả các cột phân loại chính.")

Đã hoàn thành điền thiếu 'unknown' cho tất cả các cột phân loại chính.


## **Thống kê kiểm tra dữ liệu thiếu**

In [17]:
# Xác định số lượng giá trị thiếu và tỷ lệ của các feature
df_cols = pd.DataFrame({'Count Missing': df.isna().sum(),
                        'Percent Missing': df.isnull().sum()*100/df.shape[0]})
df_cols

,Count Missing,Percent Missing
source_name,0,0.000000
event_date,0,0.000000
event_title,0,0.000000
event_description,0,0.000000
location_description,0,0.000000
location_accuracy,2,0.018127
landslide_category,0,0.000000
landslide_trigger,0,0.000000
landslide_size,9,0.081573
landslide_setting,0,0.000000


## **Lưu dữ liệu đã xử lý**


In [ ]:
df.to_csv(cf.PROCESSED_DATA, index=False, encoding='utf-8-sig')
print(f"Đã lưu dữ liệu sạch tại: {cf.PROCESSED_DATA}")

Đã lưu dữ liệu sạch tại: ..\data\processed\Global_Landslide_Processed.csv
